In [7]:
import sys
from flask import Flask, render_template,jsonify,request
from flask_cors import CORS
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpld3

In [8]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///db/inpatient.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

In [9]:
Base = automap_base(metadata=db.metadata)
engine = db.get_engine()
Base.prepare(engine, reflect=True)
Inpatient = Base.classes.inpatient
Drg = Base.classes.drg

In [43]:
#i = '872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC'
#i = '193 - SIMPLE PNEUMONIA & PLEURISY W MCC'
#i = '194 - SIMPLE PNEUMONIA & PLEURISY W CC'
#i = '291 - HEART FAILURE & SHOCK W MCC'
#i = '292 - HEART FAILURE & SHOCK W CC'
#i = '392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC'
#i = '470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC'
#i = '690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC'
#i = '871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W MCC'
i = '872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV >96 HOURS W/O MCC'

In [44]:
def sanitize(data):
    if type(data).__name__ != 'float':
        return (float( data.replace(",","")))
    else:
        return data

In [45]:

fils = []
sel = [Drg.drg_description, Inpatient.hrr_description, Inpatient.average_covered_charges, Drg.weights, Inpatient.total_discharges]
response = db.session.query(*sel).filter(Inpatient.drg_definition == Drg.drg_definition).filter_by(drg_definition = i)

for r in response:
    rec = {'drg_description': r[0], 'hrr_description': r[1], 'avg_covered_charges': r[2], 'weights': r[3], 'total_discharges':r[4]}
    #print(rec)
    #del rec['_sa_instance_state']
    fils.append(rec)

df = pd.DataFrame(fils)

df['total_discharges'] = df['total_discharges'].apply(sanitize)

summary = pd.pivot_table(df, index='hrr_description', values=['avg_covered_charges', 'weights', 'total_discharges']).reset_index()
#print(summary.head())


fig, ax = plt.subplots(figsize= (15, 10))

scatter = ax.scatter(summary['total_discharges'], summary['avg_covered_charges'], s=(summary['weights']*10)**2, c="#18bc9C", edgecolors="k", alpha=0.5, cmap=plt.cm.jet)  
ax.set_title("Medicare Charges vs Patient Discharges for " + i)
ax.set_xlabel("Average Number of Discharges per Provider in Hospital Referral Region (HRR)")
ax.set_ylabel("Average Covered Medicare Charges per Procedure ($)")
ax.grid(color="white", linestyle="-")
ax.set_facecolor('#EEEEEE')

labels = np.array(summary['hrr_description'])
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
mpld3.plugins.connect(fig, tooltip)

mpld3.display()
print(mpld3.fig_to_html(fig))
    



<style>

</style>

<div id="fig_el396419172595773287058747006"></div>
<script>
function mpld3_load_lib(url, callback){
  var s = document.createElement('script');
  s.src = url;
  s.async = true;
  s.onreadystatechange = s.onload = callback;
  s.onerror = function(){console.warn("failed to load library " + url);};
  document.getElementsByTagName("head")[0].appendChild(s);
}

if(typeof(mpld3) !== "undefined" && mpld3._mpld3IsLoaded){
   // already loaded: just create the figure
   !function(mpld3){
       
       mpld3.draw_figure("fig_el396419172595773287058747006", {"width": 1080.0, "height": 720.0, "axes": [{"bbox": [0.125, 0.125, 0.775, 0.755], "xlim": [12.480742951015529, 210.76925704898446], "ylim": [7548.864595934509, 98653.6354040655], "xdomain": [12.480742951015529, 210.76925704898446], "ydomain": [7548.864595934509, 98653.6354040655], "xscale": "linear", "yscale": "linear", "axes": [{"position": "bottom", "nticks": 10, "tickvalues": null, "tickformat": null, "scale": "linear

In [31]:
df.count()

avg_covered_charges    2684
drg_description        2684
hrr_description        2684
total_discharges       2684
weights                2684
dtype: int64

In [ ]:
for i in 

In [8]:
#the "total_discharges" column has both strings and floats in it. pd.to_numeric has not worked
types = []
for i in df['total_discharges']:
    types.append(type(i))

test = pd.Series(types)
test.value_counts()

<class 'float'>    2613
dtype: int64

In [9]:
#this is the function I made to try to correct it, but it has not worked yet
def sanitize(data):
    if type(data).__name__ != 'float':
        return (float( data.replace(",","")))
    else:
        return data
            

df['total_discharges'] = df['total_discharges'].apply(sanitize)
df.count()

average_covered_charges      2613
average_medicare_payments    2613
average_total_payments       2613
drg_definition               2613
hrr_description              2613
lat                          2473
lon                          2473
mhi                          2473
provider_city                2613
provider_id                  2613
provider_name                2613
provider_state               2613
provider_street_address      2613
provider_zip_code            2613
total_discharges             2613
dtype: int64

In [10]:
#the "total_discharges" column has both strings and floats in it. pd.to_numeric has not worked
types = []
for i in df['total_discharges']:
    types.append(type(i))

test = pd.Series(types)
test.value_counts()

<class 'float'>    2613
dtype: int64

In [11]:
summary = pd.pivot_table(df, index='hrr_description', values=['average_covered_charges', 'total_discharges'])
summary

,average_covered_charges,total_discharges
hrr_description,,
AK - Anchorage,40288.714286,36.000000
AL - Birmingham,32201.468750,42.843750
AL - Dothan,24240.200000,43.800000
AL - Huntsville,24866.000000,45.666667
AL - Mobile,23352.000000,38.700000
AL - Montgomery,31781.800000,35.800000
AL - Tuscaloosa,24889.000000,98.000000
AR - Fort Smith,19334.250000,40.000000
AR - Jonesboro,16109.000000,61.666667


In [12]:
json = summary.reset_index().to_json(orient="records")
print(json)

[{"hrr_description":"AK - Anchorage","average_covered_charges":40288.7142857143,"total_discharges":36.0},{"hrr_description":"AL - Birmingham","average_covered_charges":32201.46875,"total_discharges":42.84375},{"hrr_description":"AL - Dothan","average_covered_charges":24240.2,"total_discharges":43.8},{"hrr_description":"AL - Huntsville","average_covered_charges":24866.0,"total_discharges":45.6666666667},{"hrr_description":"AL - Mobile","average_covered_charges":23352.0,"total_discharges":38.7},{"hrr_description":"AL - Montgomery","average_covered_charges":31781.8,"total_discharges":35.8},{"hrr_description":"AL - Tuscaloosa","average_covered_charges":24889.0,"total_discharges":98.0},{"hrr_description":"AR - Fort Smith","average_covered_charges":19334.25,"total_discharges":40.0},{"hrr_description":"AR - Jonesboro","average_covered_charges":16109.0,"total_discharges":61.6666666667},{"hrr_description":"AR - Little Rock","average_covered_charges":23864.4705882353,"total_discharges":49.411764